In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from collections import defaultdict  
import math  
import json
from sys import stdout
import pickle

In [2]:
train_path = '../../user_data/model_1/'
test_path = '../../user_data/model_1/'
header = 'model_1'
now_phase = 9

In [3]:
nn = pd.read_csv(train_path + 'nn/nn_' + header + '.csv')

In [4]:
result = pd.DataFrame()
result['item'] = nn['item']
result['item'] = result['item'].apply(lambda x: x[1:])
result['item'] = result['item'].apply(lambda x: x[:-1])
result['item'] = result['item'].apply(lambda x: x.split(','))
result['item'] = result['item'].apply(lambda x: [int(y) for y in x]) 
result['user_id'] = nn['user']



result['score'] = nn['score']
result['score'] = result['score'].apply(lambda x: x[1:])
result['score'] = result['score'].apply(lambda x: x[:-1])
result['score'] = result['score'].apply(lambda x: x.split(','))
result['score'] = result['score'].apply(lambda x: [float(y) for y in x]) 

result['score'] = result['score'].apply(lambda x: [1/(1+np.exp(-y)) for y in x])

recom_item = []

for i,row in tqdm(result.iterrows()):
    tmp_list = row['item']
    score_list = row['score']
    for j in range(len(score_list)):
        recom_item.append([ row['user_id'],tmp_list[j],score_list[j]])

recom_df = pd.DataFrame(recom_item, columns=['user_id', 'item_id', 'nn']) 

17448it [07:06, 40.87it/s]


In [5]:
file_name = 'recall_0531_addsim_addAA_RA_additemtime_addcount'
recall = pd.read_csv(train_path + 'new_recall/' + file_name + '.csv')

In [6]:
recall = pd.merge(left=recall,
                 right=recom_df,
                 how='left',
                 on=['user_id','item_id'])

In [7]:
recall.to_csv(train_path + 'new_recall/' + file_name + '_addnn.csv',index=False)

In [8]:
recall.describe()

user_id       item_id           sim     feature_0     feature_1  \
count  1.539732e+07  1.539732e+07  1.539732e+07  1.539732e+07  1.539732e+07   
mean   1.676133e+04  5.788562e+04  2.925747e-05  2.027411e+01  7.404778e+65   
std    1.012841e+04  3.450147e+04  1.280217e-04  1.005096e+02  1.101397e+69   
min    1.000000e+00  1.000000e+00  0.000000e+00  1.000000e+00 -6.170000e-01   
25%    7.869000e+03  2.738600e+04  0.000000e+00  2.000000e+00  2.522400e+00   
50%    1.641800e+04  5.766200e+04  0.000000e+00  4.000000e+00  3.716663e+03   
75%    2.532500e+04  8.832300e+04  0.000000e+00  1.000000e+01  1.496739e+09   
max    3.543700e+04  1.177210e+05  1.660000e-02  1.033000e+04  3.043282e+72   

          feature_2     feature_3     feature_4     feature_5     feature_6  \
count  1.539732e+07  1.539732e+07  1.539732e+07  1.539732e+07  1.539732e+07   
mean   5.590743e+01  1.513402e+01  1.250572e+01  4.081363e+02  1.790988e+00   
std    7.885062e+01  2.352603e+01  4.205021e+01  1.320589e+02  7.641844e+00   
min    0.000000e+00  1.000000e+00  0.000000e+00  1.200000e-03  0.000000e+00   
25%    2.780000e-01  2.000000e+00  0.000000e+00  3.349641e+02  0.000000e+00   
50%    1.998340e+01  6.000000e+00  4.010000e-02  4.567279e+02  1.000000e+00   
75%    8.303110e+01  1.800000e+01  3.937500e+00  5.088019e+02  1.000000e+00   
max    5.663452e+02  6.910000e+02  5.648029e+02  5.668159e+02  5.760000e+02   

       ...  item_count_6h  item_count_4h  item_count_2h  item_count_1h  \
count  ...   4.160674e+06   2.276642e+06  872482.000000   86299.000000   
mean   ...   1.918029e+00   1.452902e+00       1.438415       1.040255   
std    ...   3.905341e+00   1.176968e+00       3.176910       0.207569   
min    ...   1.000000e+00   1.000000e+00       1.000000       1.000000   
25%    ...   1.000000e+00   1.000000e+00       1.000000       1.000000   
50%    ...   1.000000e+00   1.000000e+00       1.000000       1.000000   
75%    ...   2.000000e+00   2.000000e+00       1.000000       1.000000   
max    ...   1.600000e+02   4.600000e+01     122.000000       4.000000   

             phrase         count  diff_from_last  diff_from_next  \
count  1.539732e+07  1.539732e+07    1.395672e+07    1.386128e+07   
mean   4.594267e+00  3.496562e+01    6.940745e+00   -6.933601e+00   
std    2.853970e+00  7.900975e+01    1.489547e+01    1.487798e+01   
min    0.000000e+00  0.000000e+00    0.000000e+00   -2.990000e+02   
25%    2.000000e+00  4.000000e+00    0.000000e+00   -7.000000e+00   
50%    5.000000e+00  1.300000e+01    4.000000e+00   -4.000000e+00   
75%    7.000000e+00  3.500000e+01    7.000000e+00    0.000000e+00   
max    9.000000e+00  2.477000e+03    2.990000e+02    0.000000e+00   

       diff_from_max            nn  
count   1.539732e+07  1.539732e+07  
mean    2.832597e+01  4.921945e-02  
std     6.925535e+01  1.721346e-01  
min     0.000000e+00  3.039263e-13  
25%     0.000000e+00  8.402831e-06  
50%     1.000000e+01  1.562678e-04  
75%     2.800000e+01  3.537030e-03  
max     2.273000e+03  1.000000e+00  

[8 rows x 32 columns]